In [1]:
#-*- coding: cp949 -*-
#-*- coding: utf-8 -*- 
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


import torch
from torch import nn, optim

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

iris= load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [2]:
print(data)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0       0.0  

https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab

In [3]:
data_set = data.iloc[:,[0,1,2,3]].values.tolist()
data_set[0]

[5.1, 3.5, 1.4, 0.2]

In [4]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self, df):
    #데이터셋의 전처리를 해주는 부분
    
    #sepal length, sepal width, petal length, petal width
        self.x_data = df.iloc[:,[0,1,2,3]].values.tolist()
        self.y_data = df.iloc[:, [4]].values.tolist()

    def __len__(self):
    #데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        return len(self.x_data)

    def __getitem__(self, idx): 
    #데이터셋에서 특정 1개의 샘플을 가져오는 함수
        x = torch.tensor(self.x_data[idx], dtype = torch.float32)
        y = torch.tensor(self.y_data[idx], dtype = torch.float32)
        return x,y
        

In [5]:
#데이터로더 확인
import torch
import torchvision.transforms as transforms

dataset = CustomDataset(data)

#batch_size 만큼 묶어서 데이터 (batch_size가 커지면 속도가 빨리짐, gpu 사용 증가)
dataLoader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

dataiter = iter(dataLoader)
feature, target = dataiter.next()

print(feature)
print(target)



tensor([[6.7000, 3.0000, 5.2000, 2.3000],
        [6.4000, 3.2000, 4.5000, 1.5000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [4.6000, 3.6000, 1.0000, 0.2000]])
tensor([[2.],
        [1.],
        [1.],
        [0.]])


In [6]:
#test, train 비율
split_ratio = 0.7
split_index = int(len(data) * split_ratio)

#train, test셋 분류
#데이터가 불균형하게 분포되어 있을 경우 사용 x
train_dataset = CustomDataset(data[:split_index])
test_dataset = CustomDataset(data[split_index:len(data)])

#데이터 로더
train_dataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

print(len(train_dataLoader))
print(len(test_dataLoader))

27
12


In [7]:
print(target)

tensor([[2.],
        [1.],
        [1.],
        [0.]])


## Normalize Input?

In [8]:
# Normalize Input?
# Scale data to have mean 0 and variance 1

In [9]:
class Net(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x 



In [10]:
input_dim=4     # how many Variables are in the dataset
hidden_dim = 25 # hidden layers
output_dim=3    # number of classes
input_dim 

4

In [45]:
# Instantiate model
model=Net(input_dim,hidden_dim,output_dim)

In [46]:
#print('W:',list(model.parameters())[0].size())
#print('b',list(model.parameters())[1].size())

Loss Function


In [47]:
criterion=nn.CrossEntropyLoss()

Optimizer

In [50]:
learning_rate=0.0001
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [51]:
for epoch in range(n_epochs):
    running_loss = 0.0 
    for x, y in train_dataLoader:
        # features, gt_target = data[0].to(device), data[1].to(device)
        features, gt_target = x, y
        # print(y)
        optimizer.zero_grad()
        
        pred_target = model(features)
        loss = criterion(pred_target, gt_target.view(-1).type(torch.LongTensor))
        # loss = loss(z, gt_target.squeeze().view(-1))
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    cost = running_loss / len(train_dataLoader)
    if epoch % 10 == 0:
        print('epoch %d, loss: %.3f' %(epoch, cost))

epoch 0, loss: 0.716
epoch 10, loss: 0.714
epoch 20, loss: 0.712
epoch 30, loss: 0.712
epoch 40, loss: 0.710
epoch 50, loss: 0.708
epoch 60, loss: 0.706
epoch 70, loss: 0.704
epoch 80, loss: 0.703
epoch 90, loss: 0.700
epoch 100, loss: 0.699
epoch 110, loss: 0.696
epoch 120, loss: 0.695
epoch 130, loss: 0.693
epoch 140, loss: 0.691
epoch 150, loss: 0.690
epoch 160, loss: 0.688
epoch 170, loss: 0.687
epoch 180, loss: 0.685
epoch 190, loss: 0.683
epoch 200, loss: 0.681
epoch 210, loss: 0.679
epoch 220, loss: 0.677
epoch 230, loss: 0.675
epoch 240, loss: 0.671
epoch 250, loss: 0.671
epoch 260, loss: 0.669
epoch 270, loss: 0.666
epoch 280, loss: 0.665
epoch 290, loss: 0.665
epoch 300, loss: 0.663
epoch 310, loss: 0.659
epoch 320, loss: 0.658
epoch 330, loss: 0.655
epoch 340, loss: 0.652
epoch 350, loss: 0.651
epoch 360, loss: 0.652
epoch 370, loss: 0.647
epoch 380, loss: 0.646
epoch 390, loss: 0.644
epoch 400, loss: 0.643
epoch 410, loss: 0.641
epoch 420, loss: 0.638
epoch 430, loss: 0.638